In [55]:
# Import necessary packages
import requests
import html2text 
    #pip install html2text
    #for more documentation: https://github.com/Alir3z4/html2text
import re
from Helpers import constants, utils, vocabulary

# Bokeh for plotting.
utils.require_package("bokeh")
import bokeh.plotting as bp
from bokeh.models import HoverTool
bp.output_notebook()

Loading BokehJS ...

In [15]:
# Pull an example opinion
response = requests.get('https://www.courtlistener.com/api/rest/v3/opinions/121158/?format=json')
example_json = response.json()
example_html = example_json["html_lawbox"]

In [20]:
# Examine the html text
print(example_html)

<div>
<center><b>536 U.S. 304 (2002)</b></center>
<center><h1>ATKINS<br>
v.<br>
VIRGINIA</h1></center>
<center>No. 00-8452.</center>
<center><p><b>United States Supreme Court.</b></p></center>
<center>Argued February 20, 2002.</center>
<center>Decided June 20, 2002.</center>
CERTIORARI TO THE SUPREME COURT OF VIRGINIA
<p><span class="star-pagination">*305</span> Stevens, J., delivered the opinion of the Court, in which O'Connor, Kennedy, Souter, Ginsburg, and Breyer, JJ., joined. Rehnquist, C. J., filed a dissenting opinion, in which Scalia and Thomas, JJ., joined, <i>post,</i> p. 321. Scalia, J., filed a dissenting opinion, in which Rehnquist, C. J., and Thomas, J., joined, <i>post,</i> p. 337.</p>
<p><span class="star-pagination">*306</span> <i>James W. Ellis</i> argued the cause for petitioner. With him on the briefs were <i>Robert E. Lee,</i> by appointment of the Court, 534 U. S. 1122, <i>Mark E. Olive,</i> and <i>Charles E. Haden.</i> </p>
<p><i>Pamela A. Rumpz,</i> Assistant Att

In [34]:
# Convert html to text
html_processor = html2text.HTML2Text()
html_processor.ignore_emphasis = True
html_processor.ignore_images = True
html_processor.escape_all = True
html_processor.single_line_breaks = True
example_text = html_processor.handle(example_html)
#example_text = html2text.html2text(example_html)
print(example_text)

536 U.S. 304 (2002)

# ATKINS  
v.  
VIRGINIA

No. 00-8452.

United States Supreme Court.

Argued February 20, 2002. Decided June 20, 2002. CERTIORARI TO THE SUPREME
COURT OF VIRGINIA

*305 Stevens, J., delivered the opinion of the Court, in which O'Connor, Kennedy, Souter, Ginsburg, and Breyer, JJ., joined. Rehnquist, C. J., filed a dissenting opinion, in which Scalia and Thomas, JJ., joined, post, p. 321. Scalia, J., filed a dissenting opinion, in which Rehnquist, C. J., and Thomas, J., joined, post, p. 337.

*306 James W. Ellis argued the cause for petitioner. With him on the briefs were Robert E. Lee, by appointment of the Court, 534 U. S. 1122, Mark E. Olive, and Charles E. Haden.

Pamela A. Rumpz, Assistant Attorney General of Virginia, argued the cause for
respondent. With her on the brief was Randolph A. Beales, Attorney General.[*]

Justice Stevens, delivered the opinion of the Court.

Those mentally retarded persons who meet the law's requirements for criminal
responsibility 

In [37]:
# Split text into words
example_words = example_text.split()
print(example_words)

['536', 'U.S.', '304', '(2002)', '#', 'ATKINS', 'v.', 'VIRGINIA', 'No.', '00-8452.', 'United', 'States', 'Supreme', 'Court.', 'Argued', 'February', '20,', '2002.', 'Decided', 'June', '20,', '2002.', 'CERTIORARI', 'TO', 'THE', 'SUPREME', 'COURT', 'OF', 'VIRGINIA', '*305', 'Stevens,', 'J.,', 'delivered', 'the', 'opinion', 'of', 'the', 'Court,', 'in', 'which', "O'Connor,", 'Kennedy,', 'Souter,', 'Ginsburg,', 'and', 'Breyer,', 'JJ.,', 'joined.', 'Rehnquist,', 'C.', 'J.,', 'filed', 'a', 'dissenting', 'opinion,', 'in', 'which', 'Scalia', 'and', 'Thomas,', 'JJ.,', 'joined,', 'post,', 'p.', '321.', 'Scalia,', 'J.,', 'filed', 'a', 'dissenting', 'opinion,', 'in', 'which', 'Rehnquist,', 'C.', 'J.,', 'and', 'Thomas,', 'J.,', 'joined,', 'post,', 'p.', '337.', '*306', 'James', 'W.', 'Ellis', 'argued', 'the', 'cause', 'for', 'petitioner.', 'With', 'him', 'on', 'the', 'briefs', 'were', 'Robert', 'E.', 'Lee,', 'by', 'appointment', 'of', 'the', 'Court,', '534', 'U.', 'S.', '1122,', 'Mark', 'E.', 'Olive,

In [56]:
# Pull some processing steps from utils file

def canonicalize_digits(word):
    if any([c.isalpha() for c in word]): return word
    word = re.sub("\d", "DG", word)
    if word.startswith("DG"):
        word = word.replace(",", "") # remove thousands separator
    return word

def canonicalize_word(word, wordset=None, digits=True):
    word = word.lower()
    if digits:
        if (wordset != None) and (word in wordset): return word
        word = canonicalize_digits(word) # try to canonicalize numbers
    if (wordset == None) or (word in wordset):
        return word
    else:
        return constants.UNK_TOKEN

def canonicalize_words(words, **kw):
    return [canonicalize_word(word, **kw) for word in words]

# Process words into tokens
token_feed = canonicalize_words(example_words)

print(token_feed)

['DGDGDG', 'u.s.', 'DGDGDG', '(DGDGDGDG)', '#', 'atkins', 'v.', 'virginia', 'no.', 'DGDG-DGDGDGDG.', 'united', 'states', 'supreme', 'court.', 'argued', 'february', 'DGDG', 'DGDGDGDG.', 'decided', 'june', 'DGDG', 'DGDGDGDG.', 'certiorari', 'to', 'the', 'supreme', 'court', 'of', 'virginia', '*DGDGDG', 'stevens,', 'j.,', 'delivered', 'the', 'opinion', 'of', 'the', 'court,', 'in', 'which', "o'connor,", 'kennedy,', 'souter,', 'ginsburg,', 'and', 'breyer,', 'jj.,', 'joined.', 'rehnquist,', 'c.', 'j.,', 'filed', 'a', 'dissenting', 'opinion,', 'in', 'which', 'scalia', 'and', 'thomas,', 'jj.,', 'joined,', 'post,', 'p.', 'DGDGDG.', 'scalia,', 'j.,', 'filed', 'a', 'dissenting', 'opinion,', 'in', 'which', 'rehnquist,', 'c.', 'j.,', 'and', 'thomas,', 'j.,', 'joined,', 'post,', 'p.', 'DGDGDG.', '*DGDGDG', 'james', 'w.', 'ellis', 'argued', 'the', 'cause', 'for', 'petitioner.', 'with', 'him', 'on', 'the', 'briefs', 'were', 'robert', 'e.', 'lee,', 'by', 'appointment', 'of', 'the', 'court,', 'DGDGDG', '

In [57]:
# Process tokens into a vocabulary - collects counts of tokens and assign wordids
# note: for now, do not limit size of the vocabulary
vocab = vocabulary.Vocabulary(token_feed, size=None)
print("Vocabulary size: {:,}".format(vocab.size))

# Print out some debugging stats
print("Most common unigrams:")
for word, count in vocab.unigram_counts.most_common(10):
    print("\"{:s}\": {:,}".format(word, count))
    
# Additional useful commands to keep in mind:
# vocab dictionary: vocab.word_to_id
# convert tokens to ids: vocab.words_to_ids(token_feed)
# convert ids to tokens: vocab.ids_to_words(array_of_ids)

Vocabulary size: 3,690
Most common unigrams:
"the": 954
"of": 572
"to": 333
"and": 268
"that": 263
"DGDGDG": 262
"a": 247
"in": 220
"is": 179
"death": 145


In [63]:
# Plot the top frequencies
words, counts = zip(*vocab.unigram_counts.most_common(20))

hover = HoverTool(tooltips=[("word", "@x"), ("count", "@top")], mode="vline")
fig = bp.figure(x_range=words, plot_width=800, plot_height=400, tools=[hover])
fig.vbar(x=words, width=0.8, top=counts, hover_fill_color="firebrick")
fig.y_range.start = 0
fig.yaxis.axis_label = "Count(w)"
bp.show(fig)